In [1]:
import pandas as pd
import numpy as np
from numpy import mean, std, absolute
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import xgboost as xgb
import scipy as sp
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score as r2
from sklearn.preprocessing import normalize

from interpret.glassbox import ExplainableBoostingRegressor
from interpret import show
from interpret import set_visualize_provider
from interpret.provider import InlineProvider
set_visualize_provider(InlineProvider())

plt.style.use('ggplot')

In [2]:
data = pd.read_excel('Data/Gasification Data_Daya.xlsx', index_col=0, header=0)
data = data.iloc[1:223,:14] 
display(list(data.columns.values))

# Drop null balues and store dataframe in dataframe 2
data=data.dropna()
print(data)

#Check Null values again after removing
print(data.isnull().values.any())
print(data.isna().values.any())

X = data.iloc[:, :9]
y = data.iloc[:, 9:]

input_columns = list(X.columns.values)
input_columns = [i.split(' [', 1)[0] for i in input_columns]
output_columns = list(y.columns.values)
output_columns = [i.split(' [', 1)[0] for i in output_columns]
print(input_columns, output_columns)

Xnorm = normalize(X, 'l2', axis=1)
ynorm = normalize(y, 'l2', axis=1)

print(Xnorm.shape, ynorm.shape)

['C [%wt db]',
 'H [%wt db]',
 'O [%wt db]',
 'Moisture [%wt]',
 'Ash [%wt db]',
 'ER [-]',
 'T [ºC]',
 'Bed material',
 'Steam/Biomass',
 'H2 [%vol N2 free]',
 'CO [%vol N2 free]',
 'CO2 [%vol N2 free]',
 'CH4 [%vol N2 free]',
 'GY [Nm3/kg daf]']

        C [%wt db] H [%wt db]  ...  CH4 [%vol N2 free] GY [Nm3/kg daf]
Sr No.                         ...                                    
2            50.00        5.7  ...            6.474820        2.500000
3            50.00        5.7  ...            7.164179        2.500000
4            50.00        5.7  ...            6.474820        2.100000
5            50.00        5.7  ...            6.474820        2.400000
6            50.00        5.7  ...            8.181818        2.100000
...            ...        ...  ...                 ...             ...
219          46.85        6.3  ...            9.706147        0.947532
220          46.85        6.3  ...            9.455065        0.975752
221          46.76       5.68  ...            9.933682        0.971784
222          46.76       5.68  ...            8.778402        0.982989
223          46.76       5.68  ...            8.534014        0.973816

[187 rows x 14 columns]
False
False
['C', 'H', 'O', 'Moisture', 'Ash', 'ER',

In [14]:
seed = 42
X_train, X_test, y_train, y_test = train_test_split(Xnorm, ynorm, 
                                                    test_size=.3, 
                                                   random_state=seed)

prediction = {} 
ebm_global = []
ebm_local = []
for idx in range(y.shape[1]):
    ebm = ExplainableBoostingRegressor(random_state=seed)
    ebm.fit(X_train, y_train[:, idx])
    prediction[idx]=ebm.predict_and_explain(X_test)
    ebm_global.append(ebm.explain_global())
    ebm_local.append(ebm.explain_local(X_test[:5], y_test[:5, idx]))
    
    
    

AttributeError: 'ExplainableBoostingRegressor' object has no attribute 'predict_and_explain'

In [7]:
show(ebm_global[1])

In [8]:
show(ebm_local[1])

In [13]:
print(prediction[0][1])

[[-6.76411479e-03  8.78197496e-04 -1.35471636e-02 ...  1.80700550e-03
   1.37420575e-03  9.39119910e-04]
 [ 2.39833418e-04  4.50319087e-03  4.85591152e-02 ... -7.51386670e-04
   2.40944493e-03 -1.46482286e-05]
 [-2.15542125e-02 -2.83655916e-02  5.15236162e-03 ...  2.75894093e-03
  -2.28653961e-03  1.56725292e-03]
 ...
 [-2.15542125e-02 -1.17381329e-03 -8.90749889e-04 ... -1.43843575e-02
   2.73586616e-03  2.44010026e-03]
 [-1.27462529e-02  3.44401030e-03 -3.74420228e-03 ...  2.75894093e-03
  -2.30276446e-03 -3.08125535e-03]
 [ 5.56216743e-03  3.33675506e-03  2.32996802e-02 ...  5.01441570e-03
   8.30971874e-04  1.43397894e-03]]
